In [1]:
import dynet as dy
import json
from glove import Glove
import numpy as np
import csv
from collections import defaultdict
from operator import itemgetter
import random


def DefaultdictInside():
    return [defaultdict(int),0]

In [2]:
vectorDim = 50 # 50 100 200 300
ngram = 3 # 3->trigram   2->bigram  1->unigram



In [3]:
# bos # begin of sentence
# eos # end of sentence
# eol # end of line
# bol # begin of line

def read_dataset(fpath):
    print("Reading dataset...")
    poems=[]
    n_grams_dict=defaultdict(DefaultdictInside)
    with open(fpath) as json_file:
        data = json.load(json_file)
        for d in data:
            if d["id"]==100:
                break
            poem = d["poem"]
            poem = poem.replace("\n"," eol bol ")
            #poem = poem.lower() -- dataset already lowercased
            #poem = poem.replace("\n"," eol ")
            poem = poem.replace("."," ")
            poem = poem.replace(":"," ")
            poem = poem.replace("?"," ")
            poem = "bos bol "+poem+" eol eos"
            #poem = "bos "+poem+" eos"
            poem = poem.split()
            poems.append([])
            for i in range(len(poem)-ngram+1):
                poems[-1].append(poem[i:i+ngram])
                
                prev_gram = poems[-1][-1][:ngram-1]
                next_gram = poems[-1][-1][-1]    
                n_grams_dict[" ".join(prev_gram)][1]+=1
                n_grams_dict[" ".join(prev_gram)][0][next_gram]+=1
            n_grams_dict["eos"][1]+=1
            
    print("Just Finished Reading dataset...")
    return poems,n_grams_dict

poems,count_dict= read_dataset('unim_poem.json')
print("Poem Count:",len(poems))
#print(poems[0])
keys = list(count_dict.keys())
unique_gram = len(keys)
print("Unique Gram:",unique_gram)
tot=0
for k,v in count_dict.items():
    tot += v[1]
print("Total Word:",tot)

# replace(".:?"," ")
# , ; ! " | yok
# ' - 're 's n't 've 'll ellemedim

Reading dataset...
Just Finished Reading dataset...
Poem Count: 100
Unique Gram: 3438
Total Word: 5123


In [4]:
def read_glove(fpath):
    print('Loading word vectors...')
    word2vec = {}
    embeds = []
    word2idx = {}
    with open(fpath, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            word2idx[word] = len(embeds)
            vec = np.asarray(values[1:], dtype='float32')
            word2vec[word] = vec
            embeds.append(vec)
            
    mean = np.array(embeds).mean(axis=0,dtype='float32')
    word2vec["mmeann"]=mean
    embeds.append(mean)
    
    print("Finished Loading word vectors...")
    return np.array(embeds),word2idx,word2vec

embedding,w2i, w2v= read_glove('glovo/glove.6B.'+str(vectorDim)+'d.txt')
#w2v["mmeann"]=embedding.mean(axis=0) # if the word doesn't occur in vocab, it will take mean value
print(embedding.shape)
print(embedding.size)

Loading word vectors...
Finished Loading word vectors...
(400001, 50)
20000050


In [5]:
def sumvec(w2v,words):
    _temp = np.zeros(50)
    for w in words:
        try:
            _temp+=w2v[w]
        except:
            _temp+=w2v["mmeann"]
    return _temp

h = 150 # HiddenUnit
m = vectorDim

EPOCH = 10

_model = dy.Model()
_pW1 = _model.add_parameters((h, m))
_pb1 = _model.add_parameters(h)
_pW2 = _model.add_parameters((unique_gram, h))
_pb2 = _model.add_parameters(unique_gram)
_trainer = dy.SimpleSGDTrainer(_model)


i=0
for epoch in range(1, EPOCH + 1):
    epoch_loss = 0.0
    for p in poems:
        for gram in p:
            x=sumvec(w2v,gram[:ngram-1])
            try:
                y=keys.index(gram[-1])
            except:
                y=random.randint(0,len(keys)-1)
            dy.renew_cg()
            x = dy.inputVector(x)
            input_layer = dy.tanh(_pW1 * x + _pb1)
            hidden_layer = _pW2 * input_layer + _pb2
            output_layer = dy.softmax(hidden_layer)
            loss = dy.pickneglogsoftmax(output_layer, y)
            epoch_loss += loss.scalar_value()
            loss.backward()
            _trainer.update()
    print("Epoch %d. loss = %f" % (epoch, epoch_loss/len(poems)))

Epoch 1. loss = 409.005031
Epoch 2. loss = 409.005073
Epoch 3. loss = 409.005001
Epoch 4. loss = 409.005056
Epoch 5. loss = 409.005049
Epoch 6. loss = 409.004937
Epoch 7. loss = 409.004971
Epoch 8. loss = 409.004933
Epoch 9. loss = 409.004919
Epoch 10. loss = 409.004867


In [19]:


close=20
satir = 4
sent = ['bos bol']

for _ in range(satir):
    while 1:
        #while sent[-1]!="eol":# and len(sent)<10:
        x=sumvec(w2v,sent[-1])
        dy.renew_cg()
        x = dy.inputVector(x)
        input_layer = dy.tanh(_pW1 * x + _pb1)
        hidden_layer = _pW2 * input_layer + _pb2
        output_layer = list(dy.softmax(hidden_layer).value())
        oen = list(enumerate(output_layer))
        oen=sorted(oen,key=itemgetter(1),reverse=True)
        rnd = random.randint(0,close-1)
        sent.append(keys[oen[rnd][0]])
        #print([sent])
        if sent[-1].split()[-1]=="eol":
            break
    
    #print("\n")

print(" ".join(sent))
    
    
#     print(" ".join(sent[1:]).replace("eol",""))
#     if(sent[-1]=="eol"):
#         sent=[sent[-2]]
        
#     else:
#         sent=[sent[-1]]
    
            
#     for near in oen[:close]:
#         print(keys[near[0]],count_dict[keys[near[0]]][1])




bos bol slabs for the morn old-time step bol families hardens a' left us the morn cares what as able couldn't understand made me you-oh you life eol in her their school hardens a' left us diminished size for these bol haply bol haply balance shake loose-lipped demagogue diminished size the black men judge and trouble of concealing slabs for balance shake wine-dark river cares what cheering crowds cries what cares what diminished size river eol river eol i'll no couldn't understand then someone made me hardens a' made me suit without as able would she made me i'll no as able hardens a' couldn't understand me still hardens a' left us as able o' weel-plac'd bol haply alone eol


In [56]:
def predict_labels(doc,b,U,d,H):
    x = encode_doc(doc)
    _h= layer1(x,H,d)
    y = b+U*_h
    
    return dy.softmax(y)


def layer1(x,H,d):
    _H = dy.parameter(H)
    _d = dy.parameter(d)
    
    return dy.tanh(_H * x + _d)

def encode_doc(doc):
    doc2=[]
    embs=[]
    for w in doc:
        try:
            doc2.append(w2i[w])
        except:
            doc2.append(w2i["mmeann"])
    for idx in doc2:
        embs.append(E[idx])
    
    return dy.esum(embs)


def do_loss(probs, next_word, w2v):
    try:
        next_word_vector = w2v[next_word]
    except:
        next_word_vector = w2v["mmeann"]

    return -dy.log(dy.pick_batch_elem(probs,next_word_vector))


In [58]:
model = dy.Model()

trainer = dy.SimpleSGDTrainer(model)

h = 100 # HiddenUnit
m = vectorDim
OutUnit = 100

H = model.add_parameters((h, m))
d = model.add_parameters(h)
U = model.add_parameters((m, h))
b = model.add_parameters(m)

E = model.add_lookup_parameters((len(w2i), HiddenDim),init =  embedding )

for p in poems:
    for gram in p:
        dy.renew_cg()
        probs = predict_labels(gram[:ngram-1],b,U,d,H)
        
        loss = do_loss(probs,gram[-1],w2v)
        loss.forward()
        loss.backward()
        trainer.update()

TypeError: only size-1 arrays can be converted to Python scalars

In [32]:
print(len(w2i))
print(E)
print(E.shape())
print(E[0])
print(E[1])

400000
LookupParameter /_4
(400000, 50)
expression 9/0
expression 10/0
